In [2]:
import sys
sys.path.append('../optimization')
import location as lc

In [3]:
import pandas as pd

In [4]:
stopsdf = pd.read_csv("../get_stops_data/b20-stops-data/b20-north-stops-data.csv")
stopsdf.head()

,StopID,Name,AtStreet,BayNo,City,Lat,Long,OnStreet,Routes,WheelchairAccess
0,51202,WB HARRISON DR NS VICTORIA DR,VICTORIA DR,N,VANCOUVER,49.210673,-123.065781,HARRISON DR,020,True
1,51203,NB VICTORIA DR FS FRASERVIEW DR,FRASERVIEW DR,N,VANCOUVER,49.212765,-123.065964,VICTORIA DR,"020, N20",True
2,51204,NB VICTORIA DR FS E 61 AVE,E 61 AVE,N,VANCOUVER,49.214832,-123.065913,VICTORIA DR,"020, N20",True
3,51205,NB VICTORIA DR FS BRIGADOON AVE,BRIGADOON AVE,N,VANCOUVER,49.216910,-123.065874,VICTORIA DR,"020, N20",True
4,51206,NB VICTORIA DR FS UPLAND DR,UPLAND DR,N,VANCOUVER,49.218398,-123.065832,VICTORIA DR,"020, N20",True


In [5]:
stops = []

# iterate through each stop and init BusStop objects
for stop in stopsdf.iloc:
    connectedness = len(stop['Routes'].split(','))
    bstop = lc.BusStop(stop['Lat'], stop['Long'], stop['StopID'],[], connectedness)
    stops.append(bstop)


In [6]:
areadf = pd.read_csv("../stats-canada-data/DA.csv")
areadf.head()

,DAuid,DApop_2016,DAtdwell_2016,DAurdwell_2016,DAarea,DAir_2016,DArplamx,DArplamy,DArplat,DArplong,PRuid,CDcode,ADAcode,DAcode,ERcode,CCScode,CSDcode,CMAuid,CTname,CTcode
0,59150307,632,273,254,0.2994,0,4.025164e+06,2.004585e+06,49.287698,-123.026031,59,15,59,307,20,22,22,933,53.02,5286
1,59150308,501,223,203,0.1095,0,4.024986e+06,2.004256e+06,49.284329,-123.026065,59,15,59,308,20,22,22,933,53.02,5286
2,59150309,745,299,269,0.1118,0,4.024664e+06,2.004196e+06,49.282483,-123.029585,59,15,59,309,20,22,22,933,53.02,5286
3,59150310,536,310,283,0.1095,0,4.024866e+06,2.003949e+06,49.281378,-123.025534,59,15,59,310,20,22,22,933,53.02,5286
4,59150311,532,199,181,0.0810,0,4.024531e+06,2.003835e+06,49.279058,-123.028871,59,15,59,311,20,22,22,933,53.02,5286


In [7]:
areas = []

# iterate through each area and init Block objects
for area in areadf.iloc:
    area = lc.Block(area['DArplat'], area['DArplong'], area['DAuid'],int(area['DApop_2016']),int(area['DAurdwell_2016']), [])
    areas.append(area)

For each stop we find neighboring areas within $x$ km

In [8]:
dist = 0.5
# find neighborhood for each stop
for stop in stops:
    stop.findNeighbors(areas, dist)

Create a data frame for stops which contains StopID, Lat, Long and Connectedness

In [9]:
stopId = [stop.id for stop in stops]
lat = [stop.lat for stop in stops]
long = [stop.long for stop in stops]
conect = [stop.connectedness for stop in stops]
d = {'stopId':stopId, 'lat':lat, 'long':long, 'connectedness':conect}
stopVizDf = pd.DataFrame(data=d)
stopVizDf.head()

,stopId,lat,long,connectedness
0,51202,49.210673,-123.065781,1
1,51203,49.212765,-123.065964,2
2,51204,49.214832,-123.065913,2
3,51205,49.216910,-123.065874,2
4,51206,49.218398,-123.065832,2


Create a data frame for all Areas which are in proximity to bus stops

In [27]:
route_areas = []
for stop in stops:
    route_areas = route_areas + stop.neighborhood

print('number of areas along the routeL:',len(route_areas))

areaId = [area.id for area in route_areas]
lat = [area.lat for area in route_areas]
long = [area.long for area in route_areas]
popul = [area.pop for area in route_areas]

d = {'areaId':areaId, 'lat':lat, 'long':long, 'pop':popul}
areaVizDf = pd.DataFrame(data=d)
areaVizDf.head()

number of areas along the routeL: 464


,areaId,lat,long,pop
0,59151209.0,49.210165,-123.069500,667
1,59151211.0,49.213535,-123.062329,584
2,59151214.0,49.208190,-123.061474,628
3,59151215.0,49.207338,-123.063235,543
4,59153799.0,49.211912,-123.062435,285


In [17]:
import geopandas
import json

In [19]:
with open('../stats-canada-data/lda_000a16a_e.json') as json_file:
    areaJson = json.load(json_file)

In [21]:
import plotly.express as px

In [29]:
fig = px.choropleth_mapbox(areaVizDf, geojson=areaJson, locations='areaId', color='pop',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=11, center = {"lat": 49.25308, "lon": -123.0687},
                           opacity=0.5,
                           labels={'pop':'Population'}
                          )

In [ ]:
fig.show()